<a href="https://colab.research.google.com/github/akhiltabjula/ComputationalMethodsForInformationSystems/blob/master/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 11/11/2020)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/INFO5731_FALL2020/blob/master/In_class_exercise/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [96]:
# Write your code here
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [42]:
data_df = pd.read_csv(r'/content/stsa-train.txt',sep = 'delimiter',header= None,names = ["review"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [43]:
data_df.head(2)

,review
0,"1 a stirring , funny and finally transporting ..."
1,0 apparently reassembled from the cutting-room...


In [44]:
data_df[['polarity','review']] = data_df["review"].str.split(" ", 1, expand=True)

In [45]:
data_df.head(2)

,review,polarity
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0


In [46]:
#data cleaning
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wn= WordNetLemmatizer()
nltk.download('wordnet')
stop_words = set(stopwords.words('english')) 
import string
import re


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [47]:
def data_clean(ip):
  ip = " ".join(i.lower() for i in ip.split(" ") if i not in string.punctuation)
  ip = "".join(i for i in ip if i.isalnum() or i == " ")
  values = re.split("\W+",ip)
  ip = [wn.lemmatize(i) for i in values if i not in stop_words]
  return ip

In [48]:
data_clean("hello world $")

['hello', 'world']

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVector = TfidfVectorizer(analyzer=data_clean)
tf_idf_vector = tfidfVector.fit_transform(data_df["review"])

In [50]:
tf_idf_df = pd.DataFrame(tf_idf_vector.toarray())
tf_idf_df.columns = tfidfVector.get_feature_names()

In [54]:
x_train,x_test,y_train,y_test = train_test_split(tf_idf_df, data_df["polarity"].values,test_size=0.2)

#Multinomial NB

In [55]:
#MultinomialNB

multiNB = MultinomialNB()
model_multinb = multiNB.fit(x_train,y_train)

In [57]:
#prediction
multiNB_output = model_multinb.predict(x_test)

#Result

In [59]:
print("Accuracy Score :", accuracy_score(multiNB_output,y_test))

Accuracy Score : 0.7680635838150289


In [60]:
print("Classification result")
print(classification_report(y_test,multiNB_output))

Classification result
              precision    recall  f1-score   support

           0       0.79      0.69      0.74       658
           1       0.75      0.84      0.79       726

    accuracy                           0.77      1384
   macro avg       0.77      0.76      0.77      1384
weighted avg       0.77      0.77      0.77      1384



In [62]:
#using kfolds

kfold = KFold(10,random_state = 7, shuffle= True)
print("cross validation score")
cross_val_score(model_multinb,x_test,y_test,cv=kfold).mean()

cross validation score


0.6943384422896466

#SVM

In [67]:
#Linear SVC

svm = LinearSVC()
#train the model

svm_model = svm.fit(x_train,y_train)

In [69]:
#predict the output
svm_output = svm_model.predict(x_test)

#Results

In [70]:
print("Accuracy Score :", accuracy_score(svm_output,y_test))

Accuracy Score : 0.7622832369942196


In [71]:
print("Classification result")
print(classification_report(y_test,svm_output))

Classification result
              precision    recall  f1-score   support

           0       0.74      0.76      0.75       658
           1       0.78      0.76      0.77       726

    accuracy                           0.76      1384
   macro avg       0.76      0.76      0.76      1384
weighted avg       0.76      0.76      0.76      1384



In [72]:
#using kfolds

kfold = KFold(10,random_state = 7, shuffle= True)
print("cross validation score")
cross_val_score(svm_model,x_test,y_test,cv=kfold).mean()

cross validation score


0.7059013658638307

#KNN 

In [74]:
knn = KNeighborsClassifier()

In [75]:
#train the model
knn_model = knn.fit(x_train,y_train)

In [78]:
#predict the output
knn_output = knn_model.predict(x_test)

#Result

In [80]:
print("Accuracy Score :", accuracy_score(knn_output,y_test))

Accuracy Score : 0.48265895953757226


In [81]:
print("Classification result")
print(classification_report(y_test,knn_output))

Classification result
              precision    recall  f1-score   support

           0       0.48      0.99      0.65       658
           1       0.78      0.02      0.04       726

    accuracy                           0.48      1384
   macro avg       0.63      0.51      0.34      1384
weighted avg       0.64      0.48      0.33      1384



In [82]:
#using kfolds

kfold = KFold(10,random_state = 7, shuffle= True)
print("cross validation score")
cross_val_score(knn_model,x_test,y_test,cv=kfold).mean()

cross validation score


0.6712543009071005

#Decision Tree

In [83]:
dtc = DecisionTreeClassifier()

In [84]:
#fit the model 
dtc_model = dtc.fit(x_train,y_train)

In [87]:
#predict the model
dtc_output = dtc_model.predict(x_test)

#Results

In [88]:
print("Accuracy Score :", accuracy_score(dtc_output,y_test))

Accuracy Score : 0.6510115606936416


In [89]:
print("Classification result")
print(classification_report(y_test,dtc_output))

Classification result
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       658
           1       0.68      0.62      0.65       726

    accuracy                           0.65      1384
   macro avg       0.65      0.65      0.65      1384
weighted avg       0.65      0.65      0.65      1384



In [90]:
#using kfolds

kfold = KFold(10,random_state = 7, shuffle= True)
print("cross validation score")
cross_val_score(dtc_model,x_test,y_test,cv=kfold).mean()

cross validation score


0.6076582212490876

#Random Forest

In [93]:
rf = RandomForestClassifier()

In [95]:
#train the model
rf_model = rf.fit(x_train,y_train)

In [106]:
rf_output = rf.predict(x_test)

#Results

In [107]:
print("Accuracy Score :", accuracy_score(rf_output,y_test))

Accuracy Score : 0.7276011560693642


In [108]:
print("Classification result")
print(classification_report(y_test,rf_output))

Classification result
              precision    recall  f1-score   support

           0       0.70      0.76      0.73       658
           1       0.76      0.70      0.73       726

    accuracy                           0.73      1384
   macro avg       0.73      0.73      0.73      1384
weighted avg       0.73      0.73      0.73      1384



In [104]:
#using kfolds

kfold = KFold(10,random_state = 7, shuffle= True)
print("cross validation score")
cross_val_score(rf_model,x_test,y_test,cv=kfold).mean()

cross validation score


0.6640131373162339

#XGBoost

In [98]:
xgb = XGBClassifier()

In [99]:
#train the model
xgb_model = xgb.fit(x_train,y_train)

In [100]:
#predict the model
xgb_output = xgb.predict(x_test)

#Results

In [101]:
print("Accuracy Score :", accuracy_score(xgb_output,y_test))

Accuracy Score : 0.6495664739884393


In [102]:
print("Classification result")
print(classification_report(y_test,xgb_output))

Classification result
              precision    recall  f1-score   support

           0       0.73      0.42      0.53       658
           1       0.62      0.86      0.72       726

    accuracy                           0.65      1384
   macro avg       0.67      0.64      0.63      1384
weighted avg       0.67      0.65      0.63      1384



In [103]:
#using kfolds
kfold = KFold(10,random_state = 7, shuffle= True)
print("cross validation score")
cross_val_score(xgb_model,x_test,y_test,cv=kfold).mean()

cross validation score


0.6163382337608174

**Summary of accuracy comparision**

1. MultinomialNB : 0.7680635838150289
2. SVM : 0.7622832369942196
3. KNN : 0.48265895953757226
4. Decision Tree : 0.6510115606936416
5. Random Forest : 0.7276011560693642
6. XGBoost : 0.6495664739884393